Load in packages

In [1]:
import pandas as pd
import datetime
import numpy as np
from sklearn.preprocessing import LabelEncoder
from missingpy import MissForest
import multiprocessing
from sklearn.preprocessing import MinMaxScaler

# This goes through the first step in collecting and combining data
Note, some of the packages were too large to load on Github, so they will not be included in the folder.

Process training data

In [ ]:
# Read in processed training data
print('Read in processed training data')
#proc_train_df = pd.read_csv('training_data_woEncoding_threshold_0.8.csv')

# Read in unprocessed training data
print('Read in unprocessed training data')
weather_train_fn = 'ashrae-energy-prediction/weather_train.csv'
weather_train_df = pd.read_csv(weather_train_fn)
train_fn = 'ashrae-energy-prediction/train.csv'
train_df = pd.read_csv(train_fn)

building_fn ='ashrae-energy-prediction/building_metadata.csv' 
building_df = pd.read_csv(building_fn)

# Combine data
train_df = pd.merge(building_df, train_df, on='building_id')
train_df = pd.merge(weather_train_df, train_df, on=['site_id', 'timestamp'])

# Feature Engineering
datetimeobject = train_df.timestamp.apply(lambda x:datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
train_df['WeatherYear'] = [i.year for i in datetimeobject]
train_df['WeatherMonth'] = [i.month for i in datetimeobject]
train_df['WeatherDay'] = [i.day for i in datetimeobject]
train_df['WeatherHour'] = [i.hour for i in datetimeobject]
train_df['WeatherWeekend'] = [1 if i > 4 else 0  for i in datetimeobject.weekday()]
train_df = train_df.drop(['timestamp'], axis=1)

print("Finished feature engineering.")
train_df.to_csv('train_df_step1.csv', index=False)

Process the rest of the data

In [ ]:
# Read in train data    
train_df = pd.read_csv('train_df_step1.csv', low_memory=True)
nearlyComplete = train_df.isnull().sum(1) < 5
train_df = train_df[nearlyComplete == True]
train_df = train_df[train_df['meter_reading'] != np.inf]
train_df = train_df.sample(frac=0.1, random_state = 0)

# Assign random variables needed
print('Assign random variables needed')
features = ['air_temperature', 'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr', 'wind_speed',
            'wind_direction', 'sea_level_pressure', 'square_feet', 'year_built', 
            'WeatherYear', 'WeatherMonth', 'WeatherDay', 'WeatherHour']
cat_vars = ['site_id', 'primary_use', 'building_id', 'meter']

# Make test data
print('Make test data')
weather_test_fn = 'ashrae-energy-prediction/weather_test.csv'
weather_test_df = pd.read_csv(weather_test_fn)
test_fn = 'ashrae-energy-prediction/test.csv'
test_df = pd.read_csv(test_fn)
building_fn ='ashrae-energy-prediction/building_metadata.csv' 
building_df = pd.read_csv(building_fn)
test_df = pd.merge(building_df, test_df, on='building_id')
test_df = pd.merge(weather_test_df, test_df, on=['site_id', 'timestamp'])

# Feature engineering
print('Feature engineering')
datetimeobject = test_df.timestamp.apply(lambda x:datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
test_df['WeatherWeekend'] = [1 if i.weekday() > 4 else 0  for i in datetimeobject]
test_df['WeatherYear'] = [i.year for i in datetimeobject]
test_df['WeatherMonth'] = [i.month for i in datetimeobject]
test_df['WeatherDay'] = [i.day for i in datetimeobject]
test_df['WeatherHour'] = [i.hour for i in datetimeobject]
test_df = test_df.drop(['timestamp'], axis=1)

# Drop columns not found in processed train
print('Drop columns not found in processed train')
dropCols = [col for col in test_df.columns if col not in proc_train_df.columns]
dropCols.remove('row_id')
test_df = test_df.drop(dropCols, axis=1)
dropCols = [col for col in train_df.columns if col not in proc_train_df.columns]
train_df = train_df.drop(dropCols, axis=1)

# Combine Data
print('Combine Data')
comb_df = pd.concat([train_df, test_df])
del building_df
del weather_test_df

# Write to a file
comb_df.to_csv('combined.csv', chunksize=1000)


# This section goes through the subsequent preprocessing steps of the entire data: normalization and imputation

In [ ]:
# Read in combined data
print('Read in combined data')
comb_df = pd.read_csv('combined.csv')

# Assign random variables needed
print('Assign random variables needed')
features = ['air_temperature', 'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr', 'wind_speed',
            'wind_direction', 'sea_level_pressure', 'square_feet', 'year_built', 
            'WeatherYear', 'WeatherMonth', 'WeatherDay', 'WeatherHour']
cat_vars = ['site_id', 'primary_use', 'building_id', 'meter']

# Fix Primary Use
print('Fix Primary Use')
rowVals = list(comb_df['primary_use'])
newRowVals = [c.replace('/', '') for c in rowVals]
comb_df['primary_use'] = newRowVals
use_encoder = LabelEncoder()
comb_df['primary_use'] = use_encoder.fit_transform(comb_df['primary_use'])

# Scramble rows
comb_df = comb_df.sample(frac=1, random_state=100)

print('Imputation steps')
# Find rows not missing anything and use to impute
complete = comb_df.isnull().sum(1) < 2

# Impute set up variables
replaceCols = features + cat_vars
cat_idx = [i for i, e in enumerate(replaceCols) if e in cat_vars]

# Impute
imp = MissForest(max_iter=3)
imp.fit(comb_df[replaceCols], cat_vars=cat_idx)
print('Finished impute fit.')

def parallelize_dataframe(df, func, n_cores=4):
    df_split = np.array_split(df, n_cores)
    pool = multiprocessing.Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

def func(df):
    values = imp.transform(df[replaceCols])
    df[replaceCols] = values
    return df

comb_df = parallelize_dataframe(comb_df, func, n_cores=4)

# Reverse encode
print('Reverse Encode')
useList = [int(i) for i in comb_df['primary_use']]
comb_df['primary_use'] = use_encoder.inverse_transform(useList)

# Print output
print('Finished impute transform.')
comb_df.to_csv('imputed.csv')

# Normalize
print('Normalize')
min_max_scaler = MinMaxScaler()
values = min_max_scaler.fit_transform(comb_df[features])
comb_df[features] = values
print('Finished normalization.')
comb_df.to_csv('normalized.csv', chunksize=1000)

# Separate Train from test data
print('Separate Train from test data and finalize documents')
test_df = comb_df.loc[comb_df['meter_reading'].isna()]
train_df = comb_df.loc[comb_df['row_id'].isna()]
test_df = test_df.drop(['meter_reading'], axis = 1)
train_df = train_df.drop(['row_id'], axis = 1)

# Print to file
train_df.to_csv('train.csv', chunksize=1000)
test_df.to_csv('test.csv', chunksize=1000)


Read in combined data
Assign random variables needed
Fix Primary Use
